In [166]:
import sys
sys.path.append('/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2')
from imports import *


In [167]:

class NN:

    def __init__(self, n_input = 784, n_hidden = 30, n_out = 10, epoch = 30, mini_b = 20, l_rate = 3):
        self.feature_train = []
        self.feature_test = []
        self.target_train = []
        self.target_test = []

        self.N_input = n_input
        self.N_hidden = n_hidden
        self.N_out = n_out
        
        self.epoch = epoch 
        self.minibatch = mini_b
        self.l_rate = l_rate
        self.layers = 3

        self.parameters = self.get_parameters()


    def arrange_data(self, train_file, test_file):
        
        train_set = self.open_data(train_file)
        test_set = self.open_data(test_file)

        self.target_train, self.feature_train = self.split_data(train_set)
        self.target_test, self.feature_test = self.split_data(test_set)

    def split_data(self, data):
        #seperates a data set into the targets and features
        
        label_set = data["label"]
        target_set = [] 
        
        for row in label_set:
            new_row = self.to_matrix_form(row)
            target_set.append(new_row)   

        df = pd.DataFrame(data)
        feature_set = (df.drop('label', axis=1)/255).astype('float64')

        return np.array(target_set), np.array(feature_set)

    def to_matrix_form(self, val, size = 10):

        arr = np.zeros(size, dtype='float64')
        arr[val] = 1.

        return arr

    def open_data(self, file):
        data_set = pd.read_csv(file)
        return data_set

    def get_parameters(self):

        parameters = {
            'W1': np.random.normal(0.0, 1.0, (self.N_hidden, self.N_input)),
            'W2': np.random.normal(0.0, 1.0, (self.N_out, self.N_hidden)),

            'B1': np.random.normal(0.0, 1.0, (self.N_hidden, )) ,
            'B2': np.random.normal(0.0, 1.0, (self.N_out, ))
            
            }       

        return parameters

    def forward(self, feature_train):
        
        parameters = self.parameters
        parameters['A0'] = feature_train

        #input layer to hidden layer    
        parameters['Z1'] = np.dot(parameters['W1'], parameters['A0']) + parameters['B1']
        parameters['A1'] = self.sigmoid(parameters['Z1'])


        #hidden layer to the output layer
        parameters['Z2'] = np.dot(parameters['W2'], parameters['A1']) + parameters['B2']
        parameters['A2'] = self.sigmoid(parameters['Z2'])

        return parameters['A2'], parameters['A1']

    def backprop(self, feature, output, hidden):
        parameters = self.parameters
        new_parameters = {}

        error = 2 * (output - feature) / output.shape[0] * self.sigmoid(parameters['Z2'], True)
        new_parameters['W2'] = np.outer(error, parameters['A1'])
        #new_parameters['B2'] = parameters['B2'] 

        error = np.dot(parameters['W2'].T, error) * self.sigmoid(parameters['Z1'], True)
        new_parameters['W1'] = np.outer(error, parameters['A0'])
        #new_parameters['B1'] = parameters['B1'] 

        return new_parameters

    def sigmoid(self, z, deriv = False):
        if deriv:
            return (np.exp(-z))/((np.exp(-z)+1)**2)
        return 1.0 / (1.0 + np.exp(-z))

    def softmax(self, z, deriv = False):
       
        exps = np.exp(z - z.max())
        if deriv:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis = 0))
        
        return exps / np.sum(exps, axis = 0)

    def train_net(self, feat_train, target_train, feat_test, target_test):

        for i in range(self.epoch):
            for j in range(len(feat_train)):
                output, hidden = self.forward(feat_train[j])
                new_parameters = self.backprop(target_train[j], output, hidden)
                self.update_parameters(new_parameters)

            accuracy = self.accuracy_calc(feat_test, target_test)
            print('Epoch: {0}, Accuracy: {1}'.format(i+1, accuracy))

    def update_parameters(self, new_parameters):

        for key, value in new_parameters.items():
            self.parameters[key] -= self.l_rate * value

                
    def accuracy_calc(self, f_test, t_test):
        predictions = []

        for i in range(len(f_test)):
            output, hidden = self.forward(f_test[i])
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(t_test[i]))

        return np.mean(predictions)

    def driver(self, train_file, test_file):
        self.arrange_data(train_file, test_file)
        self.train_net(self.feature_train, self.target_train, self.feature_test, self.target_test)



In [168]:

####################### USER SETUP AREA ####################### 

n_input = 784
n_hidden = 30
n_out = 10
epoch = 30
mini_b = 20
l_rate = 3
    
####################### END OF USER SETUP #######################

nn = NN(n_input, n_hidden, n_out, epoch, mini_b, l_rate)
nn.driver("fashion-mnist_train.csv.gz", "fashion-mnist_test.csv.gz")


Epoch: 1, Accuracy: 0.7394
Epoch: 2, Accuracy: 0.761


KeyboardInterrupt: 